In [4]:
import cv2
from ultralytics import YOLO

In [5]:
model = YOLO("yolov8x.pt",task="detect")
model.eval()

Ultralytics YOLOv8.0.195  Python-3.9.13 torch-2.1.0+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
engine\trainer: task=detect, mode=train, model=yolov8x.pt, data=coco8.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, opti

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0, 16, 17, 20, 25, 58])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000273FFC033A0>
fitness: 0.8089600784632034
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.66344,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,      0.6965,     0.80957,     0.79007,     0.79007,      0.7799,     0.79007,     0.79007,     0.79007,
           0.79007,       0.995,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007,     0.79007, 

In [7]:
colors = [(0,255,0),(0,0,255),(255,0,0)]
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the video capture
    ret, frame = cap.read()

    # Perform inference using the YOLOv8 model
    results = model(frame, agnostic_nms = True)[0]

    if not results:
        continue
    
    for result in results:
        detection_count = result.boxes.shape[0]

        for i in range(detection_count):
            cls = int(result.boxes.cls[i].item())
            name = result.names[cls]
            confidence = float(result.boxes.conf[i].item())
            bounding_box = result.boxes.xyxy[i].cpu().numpy()

            x = int(bounding_box[0])
            y = int(bounding_box[1])
            width = int(bounding_box[2] - x)
            height = int(bounding_box[3] - y)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


0: 480x640 1 person, 1339.6ms
Speed: 4.5ms preprocess, 1339.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)



person


0: 480x640 1 person, 1 refrigerator, 1611.2ms
Speed: 2.4ms preprocess, 1611.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)



person
refrigerator


0: 480x640 1 person, 1 refrigerator, 1701.9ms
Speed: 1.1ms preprocess, 1701.9ms inference, 8.8ms postprocess per image at shape (1, 3, 480, 640)



person
refrigerator


0: 480x640 1 person, 1 refrigerator, 1714.9ms
Speed: 4.2ms preprocess, 1714.9ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)



person
refrigerator


KeyboardInterrupt: 